In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for data visualization
import seaborn as sns # for statistical data visualization
import ast



In [ ]:
data = '/Users/jj/Documents/COLLEGE_DOCS/CASME2/Computed_LBP_Features_Not_Including_Others_Emotion'
df = pd.read_csv(data)

print(type(df['Computed LBP'][0]))

df['Computed LBP'] = df['Computed LBP'].apply(lambda x: list(map(float, x.split(','))) if pd.notnull(x) else x)
df['Computed LBP'] = df['Computed LBP'].apply(lambda x: np.array(x).flatten() if isinstance(x, list) else x)
print(type(df['Computed LBP'][0]))


In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
col_names = df.columns

col_names

In [ ]:

# X = df.drop(['Subject', 'Filename', 'OnsetFrame', 'ApexFrame', 'OffsetFrame',
#        'Action Units', 'Estimated Emotion'], axis=1)
# y = df['Estimated Emotion']

# X = np.array(X)

X = df['Computed LBP'].to_list()
X = np.array(X)

y = df['Estimated Emotion']


print(df.at[0, 'Computed LBP'])
# print(df['Computed LBP'])
print(df.at[0, 'Estimated Emotion'])
print(X.shape)



In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# df['Computed LBP'] = df['Computed LBP'].apply(convert_to_array)
# X_test['ApexFrame'] = X_test['ApexFrame'].str.strip("/")
# X_train['ApexFrame'] = X_train['ApexFrame'].str.strip("/")
# X_train['ApexFrame'] = np.where(X_train['ApexFrame'] == '', np.nan, X_train['ApexFrame'])

# Convert 'ApexFrame' to float, replacing NaN values with 0 (or any other desired value)
# df['Computed LBP'] = df['Computed LBP'].apply(lambda x: ' '.join(x))
# df['Computed LBP'] = df['Computed LBP'].apply(lambda x: ' '.join(x))
# X_train['Computed LBP'] = pd.to_numeric(X_train['Computed LBP'])
# X_test['Computed LBP'] = pd.to_numeric(X_test['Computed LBP'])
# X_train.flatten()


In [ ]:
from pandas import unique


print(X_test['Computed LBP'].unique())
print(X_train['Computed LBP'].unique())


In [ ]:
X_train.shape, X_test.shape
y_train.shape, y_test.shape

In [ ]:
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score
import joblib


# Classifier initialization
svm_classifier = SVC(kernel='linear', C=1.0)
svm_classifier = SVC(probability=True)

# Training the Classifier
svm_classifier.fit(X_train, y_train)
joblib.dump(svm_classifier, '/Users/jj/Documents/COLLEGE_DOCS/CASME2/SVM_Classifier_Model/SVM_Model_withProba')


In [ ]:
trained_SVM_Model = joblib.load('/Users/jj/Documents/COLLEGE_DOCS/CASME2/SVM_Classifier_Model/SVM_Model_withProba')

# Test prediction using Test sample 
predictions = trained_SVM_Model.predict(X_test)
# predictions = svm_classifier.predict(X_test)

print(predictions)

accuracy = accuracy_score(y_test, predictions)
print (f"Accuracy: {accuracy * 100: .2f}%")

In [ ]:
import cv2
from skimage import feature
import numpy as np
from sklearn.svm import SVC
import joblib

def compute_lbp(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp = feature.local_binary_pattern(gray, P=8, R=1, method="uniform")

    lbp_features_flat = lbp.flatten()[:90]

    return lbp_features_flat

def classifyME(imagePath, svmModelPath):
    image = cv2.imread(imagePath)

    lbpfeatures = compute_lbp(image)

    if len(lbpfeatures) != 90:
        print(f"Error: Expected 90 features, but got {len(lbpfeatures)} features.")
        return None

    lbpfeatures = np.array([lbpfeatures])

    trained_SVM_Model = joblib.load(svmModelPath)

    prediction = trained_SVM_Model.predict(lbpfeatures)

    return prediction


imagePathtoClassify = '/Users/jj/Documents/COLLEGE_DOCS/CASME2/Extracted_ApexFrames/EP06_07_frame225.jpg'
svm_model_path = '/Users/jj/Documents/COLLEGE_DOCS/CASME2/SVM_Classifier_Model/SVM_Model_ApexFrameV6_37percent.joblib'

predictedME = classifyME(imagePathtoClassify, svm_model_path)

print(f"Micro-Expression Present is: {predictedME}")
